# Answer Key to Creating Features Quiz

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
import re

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [5]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [6]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [7]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [8]:
df.where(df.Id == 1112).show()

+--------------------+----+--------------------+--------------------+------+--------------------+----------+
|                Body|  Id|                Tags|               Title|oneTag|               words|BodyLength|
+--------------------+----+--------------------+--------------------+------+--------------------+----------+
|<p>I submitted my...|1112|iphone app-store ...|iPhone app releas...|iphone|[p, i, submitted,...|        63|
+--------------------+----+--------------------+--------------------+------+--------------------+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [9]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [11]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words2", pattern="\\W")
df = regexTokenizer.transform(df)
df = df.withColumn("DescLength", body_length(df.words2))

In [12]:
df.where(df.Id == 5123).collect()

[Row(Body="<p>Here's an interesting experiment with using Git. Think of Github's ‘pages’ feature: I write a program in one branch (e.g. <code>master</code>), and a documentation website is kept in another, entirely unrelated branch (e.g. <code>gh-pages</code>).</p>\n\n<p>I can generate documentation in HTML format from the code in my <code>master</code>-branch, but I want to publish this as part of my documentation website in the <code>gh-pages</code> branch.</p>\n\n<p>How could I intelligently generate my docs from my code in <code>master</code>, move it to my <code>gh-pages</code> branch and commit the changes there? Should I use a post-commit hook or something? Would this be a good idea, or is it utterly foolish?</p>\n", Id=5123, Tags='git branch', Title='Git branch experiment', oneTag='git', words=['p', 'here', 's', 'an', 'interesting', 'experiment', 'with', 'using', 'git', 'think', 'of', 'github', 's', 'pages', 'feature', 'i', 'write', 'a', 'program', 'in', 'one', 'branch', 'e', '

# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [13]:
assembler2 = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler2.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [14]:
scaler_q1 = Normalizer(inputCol="DescVec", outputCol="DescVecNormalizer")
df = scaler_q1.transform(df)

In [15]:
df.where(df.Id == 512).collect()

[Row(Body="<p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code that HotSpot is using after it's been running for a while?</p>\n", Id=512, Tags='java optimization hotspot', Title='How can I see the code that HotSpot generates after optimizing?', oneTag='java', words=['p', 'i', 'd', 'like', 'to', 'have', 'a', 'better', 'understanding', 'of', 'what', 'optimizations', 'hotspot', 'might', 'generate', 'for', 'my', 'java', 'code', 'at', 'run', 'time', 'p', 'p', 'is', 'there', 'a', 'way', 'to', 'see', 'the', 'optimized', 'code', 'that', 'hotspot', 'is', 'using', 'after', 'it', 's', 'been', 'running', 'for', 'a', 'while', 'p'], BodyLength=46, Desc="How can I see the code that HotSpot generates after optimizing? <p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code 

# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [16]:
scaler_q2 = StandardScaler(inputCol="DescVec", outputCol="DescVecStandardScaler4", withMean=False, withStd=False)
scalerModel_q2 = scaler_q2.fit(df)
df = scalerModel_q2.transform(df)

In [17]:
df.where(df.Id == 512).collect()

[Row(Body="<p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code that HotSpot is using after it's been running for a while?</p>\n", Id=512, Tags='java optimization hotspot', Title='How can I see the code that HotSpot generates after optimizing?', oneTag='java', words=['p', 'i', 'd', 'like', 'to', 'have', 'a', 'better', 'understanding', 'of', 'what', 'optimizations', 'hotspot', 'might', 'generate', 'for', 'my', 'java', 'code', 'at', 'run', 'time', 'p', 'p', 'is', 'there', 'a', 'way', 'to', 'see', 'the', 'optimized', 'code', 'that', 'hotspot', 'is', 'using', 'after', 'it', 's', 'been', 'running', 'for', 'a', 'while', 'p'], BodyLength=46, Desc="How can I see the code that HotSpot generates after optimizing? <p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code 

# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [18]:
from pyspark.ml.feature import MinMaxScaler
scaler_q3 = MinMaxScaler(inputCol="DescVec", outputCol="DescVecMinMaxScaler")
scalerModel_q3 = scaler_q3.fit(df)
df = scalerModel_q3.transform(df)

In [19]:
df.where(df.Id == 512).collect()

[Row(Body="<p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code that HotSpot is using after it's been running for a while?</p>\n", Id=512, Tags='java optimization hotspot', Title='How can I see the code that HotSpot generates after optimizing?', oneTag='java', words=['p', 'i', 'd', 'like', 'to', 'have', 'a', 'better', 'understanding', 'of', 'what', 'optimizations', 'hotspot', 'might', 'generate', 'for', 'my', 'java', 'code', 'at', 'run', 'time', 'p', 'p', 'is', 'there', 'a', 'way', 'to', 'see', 'the', 'optimized', 'code', 'that', 'hotspot', 'is', 'using', 'after', 'it', 's', 'been', 'running', 'for', 'a', 'while', 'p'], BodyLength=46, Desc="How can I see the code that HotSpot generates after optimizing? <p>I'd like to have a better understanding of what optimizations HotSpot might generate for my Java code at run time. </p>\n\n<p>Is there a way to see the optimized code 